In [1]:
import requests
import pandas as pd

API_KEY = 'CG-ZGHLZY66rLn27kHpEeRLC4EC'
filtered_out_ids = ['weth', 'vnst-stablecoin', 'ripple', 'wrapped-solana', 'ether-fi', 'wrapped-bitcoin', 'binance-bridged-usdt-bnb-smart-chain', 'wbnb', 'jupiter-exchange-solana']

import time

def get_coin_data(api_url, retry_count=3, wait_time=60):
    headers = {'x-cg-demo-api-key': API_KEY}
    
    for attempt in range(retry_count):
        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
            data = response.json()
            return data
        except requests.exceptions.RequestException as e:
            if response.status_code == 429:
                if attempt < retry_count - 1:
                    print(f"Request throttled {api_url}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Max retry attempts reached. Skipping this request.")
            else:
                print(f"Error occurred: {e}")
                print(f"Response status code: {response.status_code}")
                print(f"Response text: {response.text}")
                break
    
    return None


url_24h = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=volume_desc&per_page=75&page=1&sparkline=false'
coins_24h = pd.DataFrame(get_coin_data(url_24h))

url_stablecoins = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&category=stablecoins'
stablecoins = pd.DataFrame(get_coin_data(url_stablecoins))
stablecoin_ids = stablecoins['id'].tolist()

coins_filtered = coins_24h[~coins_24h['id'].isin(stablecoin_ids + filtered_out_ids)]
coins_filtered['total_volume'] = pd.to_numeric(coins_filtered['total_volume'], errors='coerce')
coins_filtered = coins_filtered.sort_values(by='total_volume', ascending=False).head(40)

columns_to_drop = ['image', 'fully_diluted_valuation', 'high_24h', 'low_24h', 'price_change_24h',
                   'price_change_percentage_24h', 'market_cap_change_24h', 'market_cap_change_percentage_24h',
                   'circulating_supply', 'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
                   'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi', 'last_updated']


coins_filtered = coins_filtered.drop(columns=columns_to_drop)



/var/folders/dk/n5gn7cgn1_ddg9435cgt0r6r0000gn/T/ipykernel_96384/12010083.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coins_filtered['total_volume'] = pd.to_numeric(coins_filtered['total_volume'], errors='coerce')


In [2]:
# Fetch BTC price and 50-day SMA
url_btc_history = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=50&interval=daily'
btc_history = get_coin_data(url_btc_history)
btc_prices = [price[1] for price in btc_history['prices']]
btc_sma50 = pd.Series(btc_prices).rolling(window=50).mean().iloc[-1]
btc_price = coins_filtered[coins_filtered['symbol'] == 'btc']['current_price'].values[0]

if btc_price > btc_sma50:
    print("BTC price is above the 50-day SMA. Favorable for long positions.")
else:
    print("BTC price is below the 50-day SMA. Not favorable for long positions.")

print(f"BTC current price: {btc_price:.2f}")
print(f"BTC 50-day SMA: {btc_sma50:.2f}")

BTC price is above the 50-day SMA. Favorable for long positions.
BTC current price: 70660.00
BTC 50-day SMA: 65271.71


In [3]:
coin_histories = {}
for index, row in coins_filtered.iterrows():
    coin_id = row['id']
    url_coin_history = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=25&interval=daily'
    coin_history = get_coin_data(url_coin_history)
    coin_histories[coin_id] = coin_history
    try:
        coin_history = get_coin_data(url_coin_history)
        coin_histories[coin_id] = coin_history
    except Exception as e:
        print(f"Error occurred for {coin_id}: {str(e)}. Skipping this coin.")
        continue
    
# Remove coins with insufficient price data from coin_histories
filtered_coin_histories = {}
for coin_id, coin_history in coin_histories.items():
    if 'prices' in coin_history and len(coin_history['prices']) >= 20:
        filtered_coin_histories[coin_id] = coin_history
    else:
        print(f"Insufficient data for {coin_id}. Removing from analysis.")

# Update the coin_histories dictionary with the filtered data
coin_histories = filtered_coin_histories

Request throttled https://api.coingecko.com/api/v3/coins/filecoin/market_chart?vs_currency=usd&days=25&interval=daily. Retrying in 60 seconds...
Insufficient data for ethena. Removing from analysis.
Insufficient data for tensor. Removing from analysis.
Insufficient data for wormhole. Removing from analysis.


In [4]:
recommended_coins = []

# Calculate MA and check for cross within the last 10 days
for coin_id, coin_history in coin_histories.items():
    prices = [price[1] for price in coin_history['prices']]
    ma20 = [sum(prices[i-20:i]) / 20 for i in range(20, len(prices))]
    current_price = coins_filtered.loc[coins_filtered['id'] == coin_id, 'current_price'].values[0]
    
    cross_detected = False
    ma20_last_6_days = [sum(prices[i-20:i]) / 20 for i in range(20, len(prices))]
    prices_last_6_days = prices[-6:]
    
    for i in range(0, len(prices_last_6_days)):
        if (prices[i] > ma20_last_6_days[i]):
            cross_detected = True
            break

    if current_price > ma20[-1]:
        recommended_coins.append({'id': coin_id, 'current_price': current_price, 'ma20': ma20[-1], 'cross_detected': cross_detected})

# Create a DataFrame from the recommended_coins list
recommended_coins_df = pd.DataFrame(recommended_coins)

# Add the '24h_trading_volume' column to the DataFrame
recommended_coins_df['24h_trading_volume'] = recommended_coins_df['id'].apply(lambda x: coins_filtered.loc[coins_filtered['id'] == x, 'total_volume'].values[0])

# Rename columns for better readability
recommended_coins_df = recommended_coins_df.rename(columns={'id': 'Coin', 'current_price': 'Current Price', 'ma20': 'MA20', 'cross_detected': 'Cross Detected'})

recommended_coins_df

[69497.7631929463, 65292.31329331289, 68425.41565643054, 67708.94360930806, 62132.51737873437, 67819.05570684028, 65535.97844847657, 63509.04496523133, 64285.80279102199, 67310.98326810106, 69938.94530200244, 70082.05122475982, 69435.75040305211, 70710.19534863312, 69919.05546148612, 69702.3087473573, 71246.95144060145, 69785.92503802372, 65440.41922587159, 66123.938539074, 68542.17715279506, 67978.61023408506, 69000.6499164776, 69401.97360911008, 71624.18486236996, 71160.51293182065]
[3738.381184462065, 3514.222472178163, 3643.280347379834, 3525.8936406700236, 3171.293836639821, 3515.6884318548646, 3493.4304387798707, 3322.891277320318, 3353.371241538487, 3454.257071389558, 3588.493402189288, 3591.5485279028403, 3505.2181161826984, 3560.26151146781, 3516.096057627979, 3507.6606559474685, 3644.767598784896, 3508.2477201085417, 3274.901203544406, 3316.6829546775257, 3332.089836615912, 3320.2815889834437, 3362.838907503287, 3453.4361261792887, 3695.239391141908, 3680.9920423458907]
[183.

,Coin,Current Price,MA20,Cross Detected,24h_trading_volume
0,bitcoin,70660.000000,68369.700084,True,40205205178
1,ethereum,3642.540000,3465.870103,True,21059521092
2,dogecoin,0.198036,0.185411,False,2207190328
3,binancecoin,581.560000,577.436828,True,1729493575
4,the-open-network,6.860000,5.066955,False,833932363
5,litecoin,101.010000,96.361801,False,809689551
6,bitcoin-cash,671.400000,572.007762,False,754823537
7,shiba-inu,0.000029,0.000029,True,721583740
8,near,7.540000,6.952447,True,703478404
9,dogwifcoin,3.990000,3.390549,False,568934284


In [5]:
opened_position_coins = ['echelon-prime', 'ethereum', 'bitcoin', 'near', 
                         'sidus', 'dogecoin', 'binancecoin', 'yield-guild-games', 'litecoin', 'tron']

import pandas as pd

def should_sell(prices, ma20):
    if prices[-1] < ma20[-1]:
        return "Sell"
    else:
        return "Hold"

coin_data = []

for coin_id in opened_position_coins:
    url_coin_history = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=20&interval=daily'
    
    try:
        coin_history = get_coin_data(url_coin_history)
        prices = [price[1] for price in coin_history['prices']]
        if len(prices) >= 20:
            ma20 = [sum(prices[i-20:i]) / 20 for i in range(20, len(prices))]
            
            coin_data.append({
                'Coin': coin_id,
                'Price': prices[-1],
                'MA20': ma20[-1],
                'Action': should_sell(prices, ma20)
            })
        else:
            print(f"Insufficient data for {coin_id}. Skipping this coin.")
    except Exception as e:
        print(f"Error occurred for {coin_id}: {str(e)}. Skipping this coin.")

df = pd.DataFrame(coin_data)
print(df)

                Coin         Price          MA20 Action
0      echelon-prime     25.581344     23.034927   Hold
1           ethereum   3650.691333   3465.870103   Hold
2            bitcoin  70751.557127  68369.700084   Hold
3               near      7.564756      6.952447   Hold
4              sidus      0.012391      0.010759   Hold
5           dogecoin      0.198265      0.185411   Hold
6        binancecoin    582.182647    577.436828   Hold
7  yield-guild-games      1.353618      1.220380   Hold
8           litecoin    101.078736     96.361801   Hold
9               tron      0.122302      0.119964   Hold


In [6]:
recommended_coins_df.to_excel("reco.xlsx")